## Modern convnet architecture patterns

### Residual connections

In [22]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Rescaling, GlobalAveragePooling2D, add
from tensorflow.keras import Input, Model

**Residual block with number of filters changes**

In [14]:
inputs = Input(shape=(32, 32, 3))
x = Conv2D(32, 3, activation="relu")(inputs)

residual = x

x = Conv2D(64, 3, activation="relu", padding="same")(x) # Target block filter changed from 32 to 64

residual = Conv2D(64, 1)(residual)

x = add([x, residual])

**Residual block with target block having a max pooling layer**

In [13]:
inputs = Input(shape=(32, 32, 3))
x = Conv2D(32, 3, activation="relu")(inputs)

residual = x

x = Conv2D(64, 3, activation="relu", padding="same")(x)
x = MaxPooling2D(2, padding="same")(x) # Target block

residual = Conv2D(64, 1, strides=2)(residual)

x = add([x, residual])

**Utility function with residual connection**

In [24]:
inputs = Input(shape=(32, 32, 3))
x = Rescaling(1./255)(inputs)

def residual_block(x, filters, pooling=False):
    
    residual = x
    x = Conv2D(filters, 3, activation="relu", padding="same")(x)
    x = Conv2D(filters, 3, activation="relu", padding="same")(x)
    
    if pooling:
        x = MaxPooling2D(2, padding="same")(x)
        residual = layers.Conv2D(filters, 1, strides=2)(residual)
    
    elif filters != residual.shape[-1]:
        residual = Conv2D(filters, 1)(residual)
    x = add([x, residual])
    
    return x

x = residual_block(x, filters=32, pooling=True)
x = residual_block(x, filters=64, pooling=True)
x = residual_block(x, filters=128, pooling=False)

x = GlobalAveragePooling2D()(x)
outputs = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_15 (InputLayer)          [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 rescaling_6 (Rescaling)        (None, 32, 32, 3)    0           ['input_15[0][0]']               
                                                                                                  
 conv2d_77 (Conv2D)             (None, 32, 32, 32)   896         ['rescaling_6[0][0]']            
                                                                                                  
 conv2d_78 (Conv2D)             (None, 32, 32, 32)   9248        ['conv2d_77[0][0]']              
                                                                                            